In [1]:
from dataset.dataset import HENetdataset
from dataset.transformation import augmenter, to_tensor
from model.lightning_wraper import HENetWrapper
from model.henet import HENet

import os 
import random

import torch
from torch.utils.data import DataLoader
import pytorch_lightning as L
from pytorch_lightning.callbacks import (
    EarlyStopping,
    LearningRateMonitor,
    ModelCheckpoint,
    ModelSummary,
    StochasticWeightAveraging
)


In [2]:
path = "F:/processed/"
img_list = os.listdir(path) 
img_paths = []
for img in img_list:
    img_paths.append(path + img)

In [3]:
# Shuffle and split data

random.shuffle(img_paths) 

total_size = len(img_paths)
train_size = int(0.8*total_size)
test_size  = int(0.1*total_size) 
valid_size =  total_size - train_size - test_size

train_data = img_paths[:train_size]
test_data  = img_paths[train_size:train_size + test_size]
valid_data = img_paths[train_size + test_size:]

In [4]:
# Torch dataset
train_dataset =  HENetdataset(img_paths  = train_data, transform=augmenter)
test_dataset  =  HENetdataset(img_paths  = test_data,  transform=to_tensor)
valid_dataset =  HENetdataset(img_paths  = valid_data, transform=to_tensor)


In [5]:
# Data loader 
batch_size = 128
pwf = False
pwt = True
train_loader =  DataLoader(train_dataset, batch_size=batch_size, shuffle=True,  num_workers= 8,  persistent_workers= pwt)

valid_loader =  DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers= 8,  persistent_workers= pwf)

test_loader  =  DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers= 8,  persistent_workers= pwt)

In [6]:
torch.set_float32_matmul_precision("high")

In [7]:
# Define model
n_classes = 100
lr = 0.01
model = HENetWrapper(model=HENet(n_classes=n_classes),
                     num_classes=n_classes,
                     learning_rate=lr
                     )
torch.cuda.empty_cache()

In [8]:
# Define trainer 

training_callbacks = [
        EarlyStopping(monitor="val_loss", mode="min"),
        StochasticWeightAveraging(swa_lrs=1e-2),
        LearningRateMonitor(logging_interval="step"),
        ModelCheckpoint(
            dirpath="./output",
            save_top_k=2,
            monitor="val_loss",
            filename="HENet-{epoch:02d}-{val_loss:.4f}-{val_accuracy:.4f}",
            save_last=True,
        ),
        ModelSummary(-1)
    ]




trainer = L.Trainer(max_epochs=40, callbacks=training_callbacks)


trainer.fit(
    model=model,
    train_dataloaders=train_loader,
    val_dataloaders=valid_loader,
    ckpt_path= None
)

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Missing logger folder: f:\HENet\lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                            | Type                | Params | Mode  | In sizes          | Out sizes        
-------------------------------------------------------------------------------------------------------------------------
0  | model                           | HENet               | 11.2 M | train | [5, 3, 224, 224]  | [5, 100]         
1  | model.backbone                  | Sequential          | 11.2 M | train | [5, 3, 224, 224]  | [5, 512, 7, 7]   
2  | model.backbone.0                | Conv2d              | 9.4 K  | train | [5, 3, 224, 224]  | [5, 64, 112, 112]
3  | model.backbone.1          

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

f:\FWC\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:419: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


Epoch 5:   0%|          | 1/750 [00:02<36:56,  0.34it/s, v_num=0, train_acc=0.931, train_loss=0.187, train_f1=0.927, train_pre=0.938, train_rec=0.931, val_loss=0.143, val_accuracy=0.934, val_f1=0.928]   

f:\FWC\lib\site-packages\pytorch_lightning\trainer\call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [9]:
trainer.test(model, test_loader) 

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 94/94 [00:27<00:00,  3.42it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.9301738142967224
         test_f1            0.9247004389762878
        test_loss           0.1509121060371399
     test_precision         0.9305580854415894
       test_recall          0.9301738142967224
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.1509121060371399,
  'test_acc': 0.9301738142967224,
  'test_f1': 0.9247004389762878,
  'test_precision': 0.9305580854415894,
  'test_recall': 0.9301738142967224}]